### Importing the needed libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler #you can use minmax scaler too
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import xgboost as xgb
from imblearn.ensemble import BalancedBaggingClassifier
#Import other necessary model libraries, for this example, using Logistic Regression

### Importing the dataset

In [2]:
train_data = pd.read_csv(r"../Data/Preprocessed-Datasets/train-data-encoded.csv", header = 0)
test_data = pd.read_csv(r"../Data/Preprocessed-Datasets/test-data-encoded.csv", header = 0)
validation_data = pd.read_csv(r"../Data/Preprocessed-Datasets/validation-data-encoded.csv", header = 0)

In [3]:
train_data=train_data.drop(['Reservation-id'], axis = 1)
validation_data=validation_data.drop(['Reservation-id'], axis = 1)
test_data=test_data.drop(['Reservation-id'], axis = 1)

In [4]:
validation_data.head()

,Age,Adults,Children,Babies,Reservation_Status,Discount_Rate,Room_Rate,Expected_stay_days,Reservation_gap,Gender_F,...,Deposit_type_No Deposit,Deposit_type_Non-Refundable,Deposit_type_Refundable,Booking_channel_Agent,Booking_channel_Direct,Booking_channel_Online,Required_Car_Parking_No,Required_Car_Parking_Yes,Use_Promotion_No,Use_Promotion_Yes
0,56,2,2,0,No-Show,15,192,2,195,0,...,1,0,0,1,0,0,1,0,0,1
1,60,2,2,0,Canceled,0,187,4,175,0,...,1,0,0,0,0,1,0,1,1,0
2,58,3,1,0,Canceled,10,227,1,193,1,...,1,0,0,0,1,0,1,0,0,1
3,23,1,2,0,Check-In,25,189,2,103,1,...,0,0,1,0,1,0,1,0,0,1
4,47,1,1,0,Check-In,10,218,1,92,1,...,1,0,0,0,0,1,0,1,0,1


In [5]:
validation_data.head()

,Age,Adults,Children,Babies,Reservation_Status,Discount_Rate,Room_Rate,Expected_stay_days,Reservation_gap,Gender_F,...,Deposit_type_No Deposit,Deposit_type_Non-Refundable,Deposit_type_Refundable,Booking_channel_Agent,Booking_channel_Direct,Booking_channel_Online,Required_Car_Parking_No,Required_Car_Parking_Yes,Use_Promotion_No,Use_Promotion_Yes
0,56,2,2,0,No-Show,15,192,2,195,0,...,1,0,0,1,0,0,1,0,0,1
1,60,2,2,0,Canceled,0,187,4,175,0,...,1,0,0,0,0,1,0,1,1,0
2,58,3,1,0,Canceled,10,227,1,193,1,...,1,0,0,0,1,0,1,0,0,1
3,23,1,2,0,Check-In,25,189,2,103,1,...,0,0,1,0,1,0,1,0,0,1
4,47,1,1,0,Check-In,10,218,1,92,1,...,1,0,0,0,0,1,0,1,0,1


### Seperating the columns of categorical and quantitative data

In [6]:
data_quan_cols = ['Age','Discount_Rate', 'Room_Rate','Expected_stay_days', 
                        'Reservation_gap', 'Adults', 'Children','Babies']

In [7]:
data_cat_cols = ['Gender', 'Ethnicity', 'Educational_Level', 'Income', 'Country_region',
                 'Hotel_Type', 'Meal_Type', 'Visted_Previously',
                 'Previous_Cancellations', 'Deposit_type', 'Booking_channel',
                 'Required_Car_Parking', 'Use_Promotion']

### Scaling the quantitative variables

In [8]:
train_data_quan = train_data[data_quan_cols]
validation_data_quan = validation_data[data_quan_cols]
test_data_quan = test_data[data_quan_cols]

In [9]:
sc = StandardScaler()

In [10]:
train_data_quan_scaled = sc.fit_transform(train_data_quan)
validation_data_quan_scaled = sc.transform(validation_data_quan)
test_data_quan_scaled = sc.transform(test_data_quan)

In [11]:
train_data_quan_scaled

array([[-0.25991554, -0.22266812,  0.97691768, ..., -0.28353117,
         0.35475481, -0.61317041],
       [ 0.32820944, -1.11506055,  0.22480302, ...,  0.56644415,
         1.73987514, -0.61317041],
       [-0.1292211 , -1.11506055, -1.27942629, ...,  0.56644415,
         1.73987514, -0.61317041],
       ...,
       [ 0.32820944, -0.66886434,  0.61225603, ..., -0.28353117,
        -1.03036552,  1.13107011],
       [ 1.37376495,  0.66972431, -0.41335487, ...,  0.56644415,
         0.35475481, -0.61317041],
       [ 0.26286222, -1.11506055,  0.79458685, ...,  0.56644415,
        -1.03036552, -0.61317041]])

In [12]:
type(train_data_quan_scaled)

numpy.ndarray

### Separating the categorical variables

In [13]:
cols_to_drop = data_quan_cols.copy()
cols_to_drop.append("Reservation_Status")
cols_to_drop

['Age',
 'Discount_Rate',
 'Room_Rate',
 'Expected_stay_days',
 'Reservation_gap',
 'Adults',
 'Children',
 'Babies',
 'Reservation_Status']

In [14]:
print(cols_to_drop)

['Age', 'Discount_Rate', 'Room_Rate', 'Expected_stay_days', 'Reservation_gap', 'Adults', 'Children', 'Babies', 'Reservation_Status']


In [15]:
train_data_cat = train_data.drop(cols_to_drop, axis=1)
validation_data_cat = validation_data.drop(cols_to_drop, axis=1)
test_data_cat = test_data.drop(data_quan_cols, axis=1)
train_data_cat.head()

,Gender_F,Gender_M,Ethnicity_African American,Ethnicity_Asian American,Ethnicity_Latino,Ethnicity_caucasian,Educational_Level_College,Educational_Level_Grad,Educational_Level_High-School,Educational_Level_Mid-School,...,Deposit_type_No Deposit,Deposit_type_Non-Refundable,Deposit_type_Refundable,Booking_channel_Agent,Booking_channel_Direct,Booking_channel_Online,Required_Car_Parking_No,Required_Car_Parking_Yes,Use_Promotion_No,Use_Promotion_Yes
0,1,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,1,0,1,0,1
1,1,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,1,0,1,1,0
2,1,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,1,0,1,1,0
3,0,1,1,0,0,0,1,0,0,0,...,0,0,1,1,0,0,0,1,0,1
4,1,0,0,0,1,0,0,0,1,0,...,1,0,0,0,1,0,1,0,0,1


In [16]:
train_data_cat_array = train_data_cat.to_numpy()
validation_data_cat_array = validation_data_cat.to_numpy()
test_data_cat_array = test_data_cat.to_numpy()
train_data_cat_array.shape

(27499, 38)

### Joining the pre-processed data

In [17]:
train_data_processed = np.concatenate((train_data_quan_scaled, train_data_cat_array), axis = 1)
validation_data_processed = np.concatenate((validation_data_quan_scaled, validation_data_cat_array), axis = 1)
test_data_processed = np.concatenate((test_data_quan_scaled, test_data_cat_array), axis = 1)

In [18]:
train_data_processed[0]

array([-0.25991554, -0.22266812,  0.97691768, -0.84878606, -0.88293639,
       -0.28353117,  0.35475481, -0.61317041,  1.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  1.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
        1.        ])

In [19]:
test_data_quan_scaled.shape

(4318, 8)

In [20]:
test_data_cat_array.shape

(4318, 38)

In [21]:
validation_data_processed.shape

(2749, 46)

### Encoding the labels

In [22]:
train_y = train_data['Reservation_Status']
validation_y = validation_data['Reservation_Status']

In [23]:
le = preprocessing.LabelEncoder()

In [24]:
train_y = le.fit_transform(train_y)
validation_y = le.transform(validation_y)

In [25]:
validation_y.shape

(2749,)

In [26]:
le

LabelEncoder()

### Coding...

In [216]:
from imblearn.over_sampling import SMOTE
#0: Cancelled, 1: Check--in , 2:No Show

In [241]:
#Oversampling
strategy = {0:4134*5, 1:21240, 2:2125*9}
smote = SMOTE(sampling_strategy=strategy) 

X_smote, y_smote = smote.fit_resample(train_data_processed, train_y)
#print(Counter(y_train), Counter(y_smote))

In [242]:
#NearMiss
from imblearn.under_sampling import TomekLinks #use TomekLinks and NearMiss

strategy = {1: round(21240*0.5), 0: 4134*2, 2: 2125*4}
undersample = TomekLinks()

X_near , y_near = undersample.fit_resample(X_smote,y_smote)

In [243]:
unique_near = np.bincount(y_near)
print(unique_near)

[20661 21231 19125]


In [244]:
xg_train = xgb.DMatrix(X_near, label=y_near)
xg_test = xgb.DMatrix(validation_data_processed, label=validation_y)

In [245]:
param = {}

# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.257
param['max_depth'] = 1
param['nthread'] = -1
param['num_class'] = 3
param['max_delta_step'] = 1
param['random_state'] = 0
#param['eval_metric'] = 'merror'
#param['scale_pos_weight'] = 0.5
#param['num_parallel_tree'] = 50

In [246]:
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 100

In [247]:
bst = xgb.train(param, xg_train, num_round, watchlist, 
                callbacks=[xgb.callback.EvaluationMonitor(show_stdv=False),xgb.callback.EarlyStopping(3)])

[13:05:00] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	train-mlogloss:1.08087	test-mlogloss:1.09525
[0]	train-mlogloss:1.08087	test-mlogloss:1.09525
[1]	train-mlogloss:1.06932	test-mlogloss:1.09580
[1]	train-mlogloss:1.06932	test-mlogloss:1.09580
[2]	train-mlogloss:1.06078	test-mlogloss:1.09646
[2]	train-mlogloss:1.06078	test-mlogloss:1.09646
[3]	train-mlogloss:1.05443	test-mlogloss:1.09944


In [248]:
y_pred = bst.predict(xg_test)

In [249]:
print(classification_report(validation_y, y_pred))

              precision    recall  f1-score   support

           0       0.28      0.34      0.31       741
           1       0.62      0.35      0.45      1610
           2       0.16      0.37      0.22       398

    accuracy                           0.35      2749
   macro avg       0.35      0.36      0.33      2749
weighted avg       0.46      0.35      0.38      2749



In [250]:
xg_to_predict = xgb.DMatrix(test_data_processed)

In [251]:
y_to_predict = bst.predict(xg_to_predict)

In [252]:
y_to_predict

array([0., 2., 1., ..., 2., 1., 0.], dtype=float32)

In [253]:
dataset_pred = pd.DataFrame({'Reservation_Status': y_to_predict})

In [254]:
dataset_pred.head()

,Reservation_Status
0,0.0
1,2.0
2,1.0
3,2.0
4,1.0


In [255]:
dataset_pred.to_csv('predictions_day3_trial.csv')

### HP Tuning XGBoost

In [156]:
xg_boost_param_grid = {
    'eta' : np.linspace(1.0, 0.0001, 25), 'max_depth': np.arange(1,10), 'sampling_method': ['uniform', 'gradient_based']
    }

In [157]:
model_xg_boost_base = xgb.XGBClassifier(random_state = 0, num_class = 3, use_label_encoder=False)

In [158]:
model_xg_boost_hp = RandomizedSearchCV(model_xg_boost_base, xg_boost_param_grid, cv=5, n_iter = 100, n_jobs = -1, verbose = 1)

In [159]:
model_xg_boost_hp.fit(X_near, y_near)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


KeyboardInterrupt: 

In [ ]:
model_xg_boost_hp.best_params_

In [ ]:
model_xg_boost_hp.best_score_

### Lasso Feature Selection

In [160]:
from sklearn.feature_selection import SelectFromModel
sel_ = SelectFromModel(xgb.XGBClassifier(penalty='l1', random_state = 0, num_class = 3, use_label_encoder=False))

In [161]:
sel_.fit(X_near, y_near)

[12:00:12] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { penalty } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:00:12] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


SelectFromModel(estimator=XGBClassifier(base_score=None, booster=None,
                                        colsample_bylevel=None,
                                        colsample_bynode=None,
                                        colsample_bytree=None, gamma=None,
                                        gpu_id=None, importance_type='gain',
                                        interaction_constraints=None,
                                        learning_rate=None, max_delta_step=None,
                                        max_depth=None, min_child_weight=None,
                                        missing=nan, monotone_constraints=None,
                                        n_estimators=100, n_jobs=None,
                                        num_class=3, num_parallel_tree=None,
                                        penalty='l1', random_state=0,
                                        reg_alpha=None, reg_lambda=None,
                                        scale_pos

In [163]:
sel_.get_support()

array([False, False, False, False, False,  True, False, False, False,
       False,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True, False, False,  True, False,  True,
       False,  True,  True,  True,  True,  True, False, False, False,
        True, False, False, False, False,  True, False, False, False,
       False])

In [166]:
X_near_pd = pd.DataFrame(X_near)

In [167]:
selected_feat = X_near_pd.columns[(sel_.get_support())]
print('total features: {}'.format((X_near_pd.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
      np.sum(sel_.estimator_.coef_ == 0)))

total features: 46
selected features: 21


AttributeError: Coefficients are not defined for Booster type None

In [177]:
selected_feat

Int64Index([ 5, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 24, 26, 28, 29, 30,
            31, 32, 36, 41],
           dtype='int64')

In [173]:
selected_feat_data = X_near_pd.iloc[:,[5, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 24, 26, 28, 29, 30,31, 32, 36, 41]]

In [174]:
selected_feat_data.shape

(61016, 21)

In [ ]:
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.257
param['max_depth'] = 1
param['nthread'] = -1
param['num_class'] = 3
param['max_delta_step'] = 1

In [195]:
model_xg_boost_l1 = xgb.XGBClassifier(random_state = 0, num_class = 3, use_label_encoder=False,
                                      objective='multi:softmax', eta = 0.257, max_depth=1  )

In [196]:
model_xg_boost_l1.fit(X_near,y_near)

[12:28:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.257, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.256999999, max_delta_step=0, max_depth=1,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_class=3, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [192]:
validation_data_processed_pd = pd.DataFrame(validation_data_processed)
validation_data_l1 = validation_data_processed_pd.iloc[:, selected_feat]

In [197]:
y_pred = model_xg_boost_l1.predict(validation_data_processed)

In [198]:
print(classification_report(validation_y, y_pred))

              precision    recall  f1-score   support

           0       0.28      0.24      0.26       741
           1       0.59      0.70      0.64      1610
           2       0.11      0.06      0.07       398

    accuracy                           0.48      2749
   macro avg       0.33      0.33      0.33      2749
weighted avg       0.44      0.48      0.46      2749

